### 1.Create tables

In [ ]:
""" 
CREATE TABLE public.colab_project_participation (
    user1_id TEXT NOT NULL,
    user2_id TEXT NOT NULL,
    project_id INTEGER NOT NULL,
    is_first_project BOOLEAN DEFAULT FALSE,
    is_last_project BOOLEAN DEFAULT FALSE,
    PRIMARY KEY (user1_id, user2_id, project_id)
);
"""

In [ ]:
"""
CREATE TABLE public.colab_pairs_single_proj (
    user1_id TEXT NOT NULL,
    user2_id TEXT NOT NULL,
    project_id INTEGER NOT NULL,
    first_proj_start TIMESTAMPTZ,
    first_proj_end TIMESTAMPTZ,
    PRIMARY KEY (user1_id, user2_id)
);

"""

""" 
CREATE TABLE public.colab_pairs_multi_proj (
    user1_id TEXT NOT NULL,
    user2_id TEXT NOT NULL,
    project_ids INTEGER[] NOT NULL,
    first_proj INTEGER NOT NULL,
    last_proj INTEGER NOT NULL,
    first_proj_start TIMESTAMPTZ,
    last_proj_end TIMESTAMPTZ,
    PRIMARY KEY (user1_id, user2_id)
);

"""

### 2. Populate tables

Use Index to optimize

Populate colab_project_participation

In [ ]:
""" 
INSERT INTO public.colab_project_participation (user1_id, user2_id, project_id)
SELECT
    cp.user1_id,
    cp.user2_id,
    unnest(cp.project_ids) AS project_id
FROM public.colab_pairs_clean cp;
"""

Populate colab_pairs_single_proj and colab_pairs_multi_proj

In [ ]:
"""
INSERT INTO public.colab_pairs_single_proj (user1_id, user2_id, project_id, first_proj_start, first_proj_end)
SELECT
    cp.user1_id,
    cp.user2_id,
    cp.project_ids[1] AS project_id,
    p.start_date,
    p.end_date
FROM public.colab_pairs_clean cp
JOIN public.projects p ON cp.project_ids[1] = p.project_id
WHERE cardinality(cp.project_ids) = 1;

"""

In [ ]:
"""
INSERT INTO public.colab_pairs_multi_proj (
    user1_id,
    user2_id,
    project_ids,
    first_proj,
    last_proj,
    first_proj_start,
    last_proj_end
)
SELECT
    cp.user1_id,
    cp.user2_id,
    cp.project_ids,
    cp.project_ids[1] AS first_proj,
    cp.project_ids[array_length(cp.project_ids, 1)] AS last_proj,
    fp.start_date AS first_proj_start,
    lp.end_date AS last_proj_end
FROM public.colab_pairs_clean cp
JOIN public.projects fp ON cp.project_ids[1] = fp.project_id
JOIN public.projects lp ON cp.project_ids[array_length(cp.project_ids, 1)] = lp.project_id
WHERE cardinality(cp.project_ids) > 1;

"""

### 3.Update first/last project based on table 'projects'

Update first/last info on colab_pairs_multi_proj

Criteria: 

First project -- MIN(start_date)

Last project --MAX(end_date). 

If changing criteria, e.g. last--max(start_date) or last--max(start_date) and max(end_date): is_first_project and is_last_project in colab_project_participation should be updated as well!


In [ ]:
""" 
CREATE INDEX IF NOT EXISTS idx_colab_pairs_multi_proj_project_ids ON colab_pairs_multi_proj USING GIN(project_ids);
"""

In [ ]:
"""
WITH pair_proj_times AS (
  SELECT
    cp.user1_id,
    cp.user2_id,
    p.project_id,
    p.start_date,
    p.end_date
  FROM colab_pairs_multi_proj cp
  JOIN projects p ON p.project_id = ANY(cp.project_ids)
),
first_last_proj AS (
  SELECT
    user1_id,
    user2_id,
    (ARRAY_AGG(project_id ORDER BY start_date ASC))[1] AS first_proj,
    (ARRAY_AGG(project_id ORDER BY end_date DESC))[1] AS last_proj,
    MIN(start_date) AS first_proj_start,
    MAX(end_date) AS last_proj_end
  FROM pair_proj_times
  GROUP BY user1_id, user2_id
)
UPDATE colab_pairs_multi_proj cp
SET first_proj = fl.first_proj,
    last_proj = fl.last_proj,
    first_proj_start = fl.first_proj_start,
    last_proj_end = fl.last_proj_end
FROM first_last_proj fl
WHERE cp.user1_id = fl.user1_id AND cp.user2_id = fl.user2_id;
"""

Update first/last info on colab_project_participation

In [ ]:
"""
CREATE INDEX CONCURRENTLY IF NOT EXISTS idx_cpp_user_pair
ON colab_project_participation (user1_id, user2_id);

CREATE INDEX CONCURRENTLY IF NOT EXISTS idx_cpmp_user_pair
ON colab_pairs_multi_proj (user1_id, user2_id);
"""

In [ ]:
"""
-- single colab pairs
UPDATE colab_project_participation
SET is_first_project = TRUE,
    is_last_project = TRUE
WHERE (user1_id, user2_id) IN (
  SELECT user1_id, user2_id
  FROM colab_project_participation
  GROUP BY user1_id, user2_id
  HAVING COUNT(*) = 1
);
"""

"""
-- multi colab pairs
UPDATE colab_project_participation cpp
SET is_first_project = TRUE
FROM colab_pairs_multi_proj mp
WHERE cpp.user1_id = mp.user1_id
  AND cpp.user2_id = mp.user2_id
  AND cpp.project_id = mp.first_proj;

UPDATE colab_project_participation cpp
SET is_last_project = TRUE
FROM colab_pairs_multi_proj mp
WHERE cpp.user1_id = mp.user1_id
  AND cpp.user2_id = mp.user2_id
  AND cpp.project_id = mp.last_proj;

"""

# test result:
# SELECT 
#   is_first_project, 
#   is_last_project, 
#   COUNT(*) 
# FROM colab_project_participation
# GROUP BY is_first_project, is_last_project;
